## Multi-dimensional 0-1 knapsack problem

In [1]:
import numpy as np
from tqdm.notebook import tqdm
from icecream import ic
import math

In [2]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [3]:
MAX_STEPS = 20000

In [4]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [5]:
def valid_solution(solution):
    if solution is None:
        return False
    for kp in range(NUM_KNAPSACKS):
        if np.any(WEIGHTS[solution == kp].sum(axis = 0) > CONSTRAINTS[kp]):
            return False
    return True

In [6]:
# compute total value of items in knapsacks
def fitness(solution):
    if not valid_solution(solution):
        return 0
    total_value = 0
    for kp in range(NUM_KNAPSACKS):
        total_value += VALUES[solution == kp].sum()
    return total_value 

In [7]:
def tweak(solution):
    new_solution = np.array(solution, copy=True)
    indexes = np.random.choice(NUM_ITEMS, size=(NUM_ITEMS//max(1,(NUM_ITEMS//20))), replace=False)
    new_values = np.random.randint(-1, NUM_KNAPSACKS, size=len(indexes))
    mask = new_values == new_solution[indexes]
    new_values[mask] = (new_values[mask] + 1 - (-1)) % NUM_KNAPSACKS - 1
    new_solution[indexes] = new_values
    return new_solution


## TEST PROBLEMS

In [8]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [9]:
solution_representations = list(range(-1, NUM_KNAPSACKS)) 
current_solution = np.concatenate([np.random.choice(solution_representations, size=NUM_ITEMS//6), [-1]*(NUM_ITEMS - NUM_ITEMS//6)])
while not valid_solution(current_solution):
    current_solution = np.concatenate([np.random.choice(solution_representations, size=NUM_ITEMS//6), [-1]*(NUM_ITEMS - NUM_ITEMS//6)])
current_obj = fitness(current_solution)
best_solution = current_solution 
best_obj = current_obj 
ic(best_solution, best_obj, valid_solution(best_solution))

ic| best_solution: array([ 2,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                          -1, -1, -1])
    best_obj: np.int64(85)
    valid_solution(best_solution): True


(array([ 2,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1]),
 np.int64(85),
 True)

In [10]:
temp = 1.0

for step in tqdm(range(MAX_STEPS)):
    new_solution = tweak(current_solution)
    new_obj = fitness(new_solution)
    if new_obj >= current_obj:
        current_solution = new_solution
        current_obj = new_obj            
        best_solution = current_solution
        best_obj = current_obj
    elif new_obj == 0 or new_obj < current_obj:
        diff = new_obj - current_obj
        p = math.exp(diff / temp)

        if np.random.rand() < p:
            current_solution = new_solution
            current_obj = new_obj
    temp *= 0.99
    
ic(best_solution, best_obj, valid_solution(best_solution))           
    

  0%|          | 0/20000 [00:00<?, ?it/s]

ic| best_solution: array([-1,  1,  0,  1,  0,  0,  2,  0,  0,  2,  0,  1,  1,  1,  0,  2,  0,
                           0,  0,  2])
    best_obj: np.int64(1057)
    valid_solution(best_solution): True


(array([-1,  1,  0,  1,  0,  0,  2,  0,  0,  2,  0,  1,  1,  1,  0,  2,  0,
         0,  0,  2]),
 np.int64(1057),
 True)

In [11]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [12]:
solution_representations = list(range(-1, NUM_KNAPSACKS)) 
current_solution = np.concatenate([np.random.choice(solution_representations, size=NUM_ITEMS//6), [-1]*(NUM_ITEMS - NUM_ITEMS//6)])
while not valid_solution(current_solution):
    current_solution = np.concatenate([np.random.choice(solution_representations, size=NUM_ITEMS//6), [-1]*(NUM_ITEMS - NUM_ITEMS//6)])
current_obj = fitness(current_solution)
best_solution = current_solution 
best_obj = current_obj 
ic(best_solution, best_obj, valid_solution(best_solution))

ic| best_solution: array([ 2,  5,  4,  8,  9,  7,  4,  5,  3,  5,  9,  3,  4,  9,  2,  9, -1,
                          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
                          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])
    best_obj: np.int64(8822)
    valid_solution(best_solution): True


(array([ 2,  5,  4,  8,  9,  7,  4,  5,  3,  5,  9,  3,  4,  9,  2,  9, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
 np.int64(8822),
 True)

In [13]:
temp = 1.0

for step in tqdm(range(MAX_STEPS)):
    new_solution = tweak(current_solution)
    new_obj = fitness(new_solution)
    if new_obj >= current_obj:
        current_solution = new_solution
        current_obj = new_obj            
        best_solution = current_solution
        best_obj = current_obj
    elif new_obj == 0 or new_obj < current_obj:
        diff = new_obj - current_obj
        p = math.exp(diff / temp)

        if np.random.rand() < p:
            current_solution = new_solution
            current_obj = new_obj
    temp *= 0.99

ic(best_solution, best_obj, valid_solution(best_solution))           
    

  0%|          | 0/20000 [00:00<?, ?it/s]

ic| best_solution: array([ 2,  3,  3,  0,  4,  7,  5,  9, -1,  6,  6,  3,  8,  6,  2,  9, -1,
                           1, -1, -1, -1, -1, -1,  1,  4,  4,  5, -1,  0,  1,  7, -1, -1, -1,
                           9,  6,  9, -1, -1,  9, -1, -1, -1, -1, -1, -1,  9, -1, -1, -1, -1,
                           8,  8,  5, -1, -1,  9,  2, -1, -1, -1, -1, -1,  1, -1, -1,  5,  3,
                          -1, -1, -1, -1, -1,  1, -1, -1, -1,  5,  5, -1, -1, -1,  0, -1,  4,
                          -1,  0,  4, -1, -1, -1, -1,  4, -1,  0,  0, -1, -1, -1,  7])
    best_obj: np.int64(24071)
    valid_solution(best_solution): True


(array([ 2,  3,  3,  0,  4,  7,  5,  9, -1,  6,  6,  3,  8,  6,  2,  9, -1,
         1, -1, -1, -1, -1, -1,  1,  4,  4,  5, -1,  0,  1,  7, -1, -1, -1,
         9,  6,  9, -1, -1,  9, -1, -1, -1, -1, -1, -1,  9, -1, -1, -1, -1,
         8,  8,  5, -1, -1,  9,  2, -1, -1, -1, -1, -1,  1, -1, -1,  5,  3,
        -1, -1, -1, -1, -1,  1, -1, -1, -1,  5,  5, -1, -1, -1,  0, -1,  4,
        -1,  0,  4, -1, -1, -1, -1,  4, -1,  0,  0, -1, -1, -1,  7]),
 np.int64(24071),
 True)

In [14]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [15]:
solution_representations = list(range(-1, NUM_KNAPSACKS)) 
current_solution = np.concatenate([np.random.choice(solution_representations, size=NUM_ITEMS//6), [-1]*(NUM_ITEMS - NUM_ITEMS//6)])
while not valid_solution(current_solution):
    current_solution = np.concatenate([np.random.choice(solution_representations, size=NUM_ITEMS//6), [-1]*(NUM_ITEMS - NUM_ITEMS//6)])
current_obj = fitness(current_solution)
best_solution = current_solution 
best_obj = current_obj 
ic(best_solution, best_obj, valid_solution(best_solution))

ic| best_solution: array([89, 36, 77, ..., -1, -1, -1], shape=(5000,))
    best_obj: np.int64(418465)
    valid_solution(best_solution): True


(array([89, 36, 77, ..., -1, -1, -1], shape=(5000,)), np.int64(418465), True)

In [16]:
temp = 1.0

for step in tqdm(range(MAX_STEPS)):
    new_solution = tweak(current_solution)
    new_obj = fitness(new_solution)
    if new_obj >= current_obj:
        current_solution = new_solution
        current_obj = new_obj            
        best_solution = current_solution
        best_obj = current_obj
    elif new_obj == 0 or new_obj < current_obj:
        diff = new_obj - current_obj
        p = math.exp(diff / temp)

        if np.random.rand() < p:
            current_solution = new_solution
            current_obj = new_obj
    temp *= 0.99

ic(best_solution, best_obj, valid_solution(best_solution))           
    

  0%|          | 0/20000 [00:00<?, ?it/s]

ic| best_solution: array([89, 36, 77, ..., -1, 81, -1], shape=(5000,))
    best_obj: np.int64(924696)
    valid_solution(best_solution): True


(array([89, 36, 77, ..., -1, 81, -1], shape=(5000,)), np.int64(924696), True)